# 04 - Modelagem Preditiva
Autora: Fernanda Baptista de Siqueira  
Curso: MBA em Tecnologia para Negócios – AI, Data Science e Big Data  
Tema: Análise de Acidentes de Trânsito em Porto Alegre (2020–2024)  
Origem DataFrame: Equipe Armazém de Dados de Mobilidade - EAMOB/CIET  
https://dadosabertos.poa.br/dataset/acidentes-de-transito-acidentes (11/05/2025)  

### 1. Importa bibliotecas e funções. Carrega dados

In [1]:
from config import (
    pd, sns, plt, train_test_split, 
    classification_report, confusion_matrix,
    roc_auc_score, resumo_df,
    RocCurveDisplay, StandardScaler, RandomForestClassifier,
    OneHotEncoder, ColumnTransformer, Pipeline, SimpleImputer,
    SMOTE, LogisticRegression, XGBClassifier, classification_report,
    confusion_matrix, Pipeline, SAMIRAX,    
    PATH_CLEAN, COLS_VEICULOS
)

df = pd.read_parquet(PATH_CLEAN + "df_limpo_chuva.parquet")

ImportError: cannot import name 'SAMIRAX' from 'config' (/workspaces/acidentes-poa/notebooks/config.py)

### 2. Checa dados. Divide treino e validação

In [ ]:
resumo_df(df)

df_treino = df[df["data"] < "2025-01-01"]
df_valid = df[df["data"] >= "2025-01-01"]
print(resumo_df(df_treino))


### 3. Define variável alvo e features

In [ ]:
# Alvo: acidentes com vítimas (binário)
target = "cont_vit"

# Features: categóricas + veículos + clima
features = [
    "regiao", "dia_sem", "noite_dia", "tipo_acid",
    "hora", "mes", "chuva_hora", "chovendo"
] + COLS_VEICULOS

X_train, y_train = df_treino[features], df_treino[target]
X_valid, y_valid = df_valid[features], df_valid[target]


02. Define problema modelagem
* Alvo: acidentes com vitmas, ou gravidade
* Preditores: dados temporais, regionais, meteorológicos, cols_veiculos

### 4. Pré-processamento

In [ ]:
# Separar colunas
cat_cols = ["regiao", "dia_sem", "noite_dia", "tipo_acid"]
num_cols = ["hora", "mes", "chuva_hora", "chovendo"] + cols_veiculos

# Pré-processamento
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", categorical_transformer, cat_cols),
        ("numeric", numeric_transformer, num_cols)
    ]
)

03. Pré processamento
* seleção de features
* balanceamento (SMOTE, undersampling)
* normalização

4. Modelos candidatos
* Classificação: Regressão Logística, Random Forest, XGBoost
* Series Temporais: SARIMA, Prophet

5. Treinamento e validação
* Separação treino/teste
* Treinar modelos, guardar métricas (accuracy, recall, AUC)
* Comparar modelos

6. Interpretação
* Importância das variáveis (ex.: feature_importances_ da Random Forest)
* Gráficos de desempenho (ROC, matriz de confusão)

7. Previsão para 2025
* Usar o modelo escolhido para prever
* Comparar com dados reais de 2025 quando disponíveis.